---
* **Universidad:** UOC (Universitat Oberta de Catalunya)
* **Titulación:** Master Ciencia de Datos/TFM
* **Título:** Segmentación de imágenes utilizando Redes Generativas Antagónicas
* **Actividad**: Segmentación partes cuerpo personas con armas cortas utilizando modelos pre entrenados
* **Autor:** Miguel Ponce
* **Correo:** miguelalejoponce@uoc.edu
* **Fecha:** "Enero, 04 de 2023"
* **Salida**: ipynb
---
## Contexto: Detección de personas

* **Contexto:**
Se desea segmentar las personas y armas cortas con los modelos pre entrenados.

* **Problema:** 
Es necesrio realizar la segmentación de las personas armadas con armas cortas, para tener un conjunto de imagenes que luego serán procesadas para ser utilizadas como conjunto de datos para la red GAN.

* **Solucion :** 
Se utilizara los modelos pre entrenados de las etapas anteriores para generar un nuevo conjunto de datos. Para esto se hara uso de los conjuntos de datos tratado con super pixeles ESRGAN y RDN.
* **Entorno:** con GPU.
  

## Origen del la fuente de datos
* **RDN:** https://github.com/miguelalejo/gan-segmentation-person-gun/blob/main/data/raw/super-resolution-rdn.tar.gz

## Modelos pre entrenados
* **DETECCIÓN ARMA:**
https://drive.google.com/file/d/1xAbxae-WI2uiTPUOZvakSNhvbkNBc2sT/view

* **SEGMENTACIÓN ARMA:** https://drive.google.com/file/d/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob/view

* **SEGMENTACIÓN PERSONA:** https://github.com/de-code/python-tf-bodypix

# Deteccion Personas Sin Armas

## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [4]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-59-gfdc35b1 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.5/78.2 GB disk)


## Importar Librerias



In [5]:
import os
import glob
import time
import torch
from pathlib import Path
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_hub as hub
import logging
import torch

## Constantes

In [6]:
# Declaring Constants
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"
TEST_MODE = False
MAX_IMAGES = 100
mainPath = "./"

## Clases


In [ ]:
class DetectImage:
  def __init__(self,sourcePath,outputPath,config,modelName,threshold) -> None:
    self.cfg = config
    print(os.path.join(self.cfg.OUTPUT_DIR, modelName))
    self.cfg.MODEL.WEIGHTS = os.path.join(self.cfg.OUTPUT_DIR, modelName)
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    self.predictor = DefaultPredictor(self.cfg)
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def crop_object(self,imagePath):
    """Crops an object in an image

    Inputs:
      image: PIL image
      box: one box from Detectron2 pred_boxes
    """  
    image = Image.open(imagePath)
    im = cv2.imread(imagePath)
    outputs = self.predictor(im)
    boxes = outputs["instances"].pred_boxes
    # Select 1 box:    
    if list(boxes):
      #print(imagePath)
      #print(boxes,"boxes")
      box = list(boxes)[0].detach().cpu().numpy()         
      x_top_left = box[0]
      y_top_left = box[1]
      x_bottom_right = box[2]
      y_bottom_right = box[3]
      x_center = (x_top_left + x_bottom_right) / 2
      y_center = (y_top_left + y_bottom_right) / 2
      crop_img = image.crop((int(x_top_left), int(y_top_left), int(x_bottom_right), int(y_bottom_right)))
      #plt.figure()
      #plt.axis('off')
      #plt.imshow(crop_img) 
      #plt.show() 
      return True,crop_img,box
    else:
      return False,None,None

  def saveBox(self,box,boxPath):
    with open(boxPath, "w") as f:
      for s in box:
        f.write(str(s) +"\n")
  def readBox(self,boxPath):
    values = []
    with open(boxPath, "r") as f:
      for line in f:
        values.append(float(line.strip()))
    return values
   
  
  def processImages(self):       
    path = self.sourcePath+"/*.jpg"
    print(path)
    images = glob.glob(path)
    index = 0
    start = time.time()
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1             
      hasBox,crop_img,box = self.crop_object(imagePath)
      if hasBox:
        imageName = os.path.basename(imagePath)
        print(imageName)
        name = Path(imagePath).stem        
        boxPath = os.path.join(self.outputPath,"boxes",name+".txt")
        cropImagePath = os.path.join(self.outputPath,"crops",imageName)
        crop_img.save(cropImagePath)
        self.saveBox(box,boxPath)
        print("Time Detect Image : %f" % (time.time() - start))

## Descargar conjunto de datos de personas

In [1]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rsQh1QoFo9UAIYVvRMtibnYi5TOq1_sm' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1rsQh1QoFo9UAIYVvRMtibnYi5TOq1_sm" -O ./person-not-armed.rar && rm -rf /tmp/cookies.txt

--2023-01-09 06:45:58--  https://docs.google.com/uc?export=download&confirm=t&id=1rsQh1QoFo9UAIYVvRMtibnYi5TOq1_sm
Resolving docs.google.com (docs.google.com)... 142.250.103.139, 142.250.103.113, 142.250.103.138, ...
Connecting to docs.google.com (docs.google.com)|142.250.103.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nq3rs99253q19k115nu4vm4rf5361gnv/1673246700000/07156903365772177219/*/1rsQh1QoFo9UAIYVvRMtibnYi5TOq1_sm?e=download&uuid=e586fc82-0972-4def-a595-52d35f45b2bc [following]
--2023-01-09 06:45:58--  https://doc-0c-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nq3rs99253q19k115nu4vm4rf5361gnv/1673246700000/07156903365772177219/*/1rsQh1QoFo9UAIYVvRMtibnYi5TOq1_sm?e=download&uuid=e586fc82-0972-4def-a595-52d35f45b2bc
Resolving doc-0c-bk-docs.googleusercontent.com (doc-0c-bk-docs.googleusercontent.com)... 173.194.195.

In [3]:
!unrar x person-not-armed.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from person-not-armed.rar

Creating    person-not-armed                                          OK
Extracting  person-not-armed/000013469.jpg                                 0%  OK 
Extracting  person-not-armed/000040154.jpg                                 0%  OK 
Extracting  person-not-armed/000071686.jpg                                 0%  OK 
Extracting  person-not-armed/000111209.jpg                                 0%  OK 
Extracting  person-not-armed/000252755.jpg                                 1%  OK 
Extracting  person-not-armed/000316731.jpg                                 1%  OK 
Extracting  person-not-armed/000324793.jpg                                 1%  OK 
Extracting  person-not-armed/000338399.jpg                                 1%  OK 
Extracting  person-not-armed/000360424.jpg                                 1%

## Detectar Persona

In [8]:
!rm -rf ./detect-not-armed-humans
!python detect.py --weights yolov5s.pt --img 240 --conf 0.80 --project detect-not-armed-humans   --classes 0 --save-crop  --source /content/person-not-armed

detect: weights=['yolov5s.pt'], source=/content/person-not-armed, data=data/coco128.yaml, imgsz=[240, 240], conf_thres=0.8, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=False, classes=[0], agnostic_nms=False, augment=False, visualize=False, update=False, project=detect-not-armed-humans, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-59-gfdc35b1 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
WARNING ⚠️ --img-size [240, 240] must be multiple of max stride 32, updating to [256, 256]
image 1/350 /content/person-not-armed/000013469.jpg: 160x256 1 person, 11.8ms
image 2/350 /content/person-not-armed/000040154.jpg: 160x256 (no detections), 8.1ms
image 3/350 /content/person-not-armed/000071686.jpg: 192x256 1 person, 11.3ms
image 4/350 /content/person-not-

# Segemtancion partes del cuerpo

## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [9]:
!pip uninstall -y tensorflow
!pip uninstall -y tensorflow_decision_forests
!rm -rf build/ **/*.so

Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Successfully uninstalled tensorflow-2.9.2


In [10]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 tf_bodypix opencv-python tfjs_graph_converter matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [11]:
!python -m pip install --upgrade imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Clases

In [12]:
class BodyPix:
  def __init__(self,sourcePath,outputPath) -> None:
    #self.bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16))
    self.bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_RESNET50_FLOAT_STRIDE_16))
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def maskImage(self,imagePath):
    imageName = os.path.basename(imagePath)
    
    # load model (once)

    image = tf.keras.utils.load_img(imagePath)
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    # get prediction result
    #image = tf.keras.preprocessing.image.load_img(fullPath)    
    result = self.bodypix_model.predict_single(image_array)

    # simple mask
    mask = result.get_mask(threshold=0.75)
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-mask",imageName),
        mask)

    # colored mask (separate colour for each body part)
    colored_mask = result.get_colored_part_mask(mask)
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-colored-mask",imageName),
        colored_mask)



    poses = result.get_poses()
    image_with_poses = draw_poses(
        image_array.copy(),  # create a copy to ensure we are not modifing the source image
        poses,
        keypoints_color=(255, 100, 100),
        skeleton_color=(100, 100, 255)
    )
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-poses",imageName),
        image_with_poses
    )
    

  def processImages(self):       
    start = time.time() 
    path = self.sourcePath+"/*.jpg"
    print(path)
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1
      imageName = os.path.basename(imagePath)
      print(imageName)      
      self.maskImage(imagePath) 
      print("Time Taken Body Pix : %f" % (time.time() - start))

## Liberias BodyPix

In [13]:
import tensorflow as tf
from tf_bodypix.api import download_model, load_model, BodyPixModelPaths
from tf_bodypix.draw import draw_poses  # utility function using OpenCV

## Creacion proyecto body pix RDN

In [14]:
!rm -rf body-pix-not-armed
!mkdir body-pix-not-armed
!mkdir "body-pix-not-armed/person"
!mkdir "body-pix-not-armed/person/output-mask"
!mkdir "body-pix-not-armed/person/output-colored-mask"
!mkdir "body-pix-not-armed/person/output-poses"

## Creacion Pipeline red detección partes cuerpo RDN

In [15]:
sourcePathMask = os.path.join(mainPath,"/content/yolov5/detect-not-armed-humans/exp/crops/person")
outputPathBody = os.path.join(mainPath,"body-pix-not-armed/person")
bodyPix = BodyPix(sourcePathMask,outputPathBody)
bodyPix.processImages()

/content/yolov5/detect-not-armed-humans/exp/crops/person/*.jpg
0008681212.jpg
Time Taken Body Pix : 6.370852
008987685.jpg
Time Taken Body Pix : 6.850444
031972085.jpg
Time Taken Body Pix : 7.579869
003373011.jpg
Time Taken Body Pix : 8.069534
012626474.jpg
Time Taken Body Pix : 8.571072
003218440.jpg
Time Taken Body Pix : 9.280838
009124239.jpg
Time Taken Body Pix : 9.745411
thirteen-j-NUAazpP3KIw-unsplash.jpg
Time Taken Body Pix : 11.171457
gianandrea-villa-t4cSB-InqGY-unsplash.jpg
Time Taken Body Pix : 13.197462
gift-habeshaw-ImFZSnfobKk-unsplash.jpg
Time Taken Body Pix : 16.327263
000618382.jpg
Time Taken Body Pix : 16.371804
010468957.jpg
Time Taken Body Pix : 17.224082
001444210.jpg
Time Taken Body Pix : 17.976537
010456784.jpg
Time Taken Body Pix : 18.742410
011786011.jpg
Time Taken Body Pix : 19.591528
016056238.jpg
Time Taken Body Pix : 20.942090
036484106.jpg
Time Taken Body Pix : 21.444382
005784268.jpg
Time Taken Body Pix : 21.919116
pexels-matheus-bertelli-1906157.jpg
Time

# Merge files

## Librerias

In [17]:
import shutil

## Clases

In [24]:
class MergeImages:
  def __init__(self,sourcePathOrignal, formatOriginal, sourcePathMask, formatMask,outputPath) -> None:    
    self.sourcePathOrignal = sourcePathOrignal
    self.sourcePathMask = sourcePathMask
    self.outputPath = outputPath  
    self.formatOriginal = formatOriginal
    self.formatMask = formatMask

  def mergeImages(self,pathImageMask,pathImageReal,name):
    images = [Image.open(x) for x in [pathImageMask, pathImageReal]]
    widths, heights = zip(*(i.size for i in images))
    total_width = sum(widths)
    max_height = max(heights)
    new_im = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for im in images:
      new_im.paste(im, (x_offset,0))
      x_offset += im.size[0]    
    filePath = os.path.join(self.outputPath,name+"."+self.formatOriginal)
    new_im.save(filePath)
    print("Saved merge image as %s" % filePath)

  def processImages(self):
    path = self.sourcePathMask+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:        
      if TEST_MODE:
              if index == MAX_IMAGES:         
                break
              index = index + 1        
      imageName = os.path.basename(imagePath)
      name = Path(imageName).stem 
      pathImageMask = os.path.join(self.sourcePathMask,name+"."+self.formatMask)      
      pathImageOrignal = os.path.join(self.sourcePathOrignal,name+"."+self.formatOriginal)
      self.mergeImages(pathImageMask,pathImageOrignal,name)     
      

## Creacion proyecto merge armas segmentadas y personas

In [25]:
!rm -rf data-set-person-not-armed
!mkdir data-set-person-not-armed

In [26]:
pathOriginal ="/content/yolov5/detect-not-armed-humans/exp/crops/person"
pathColoredMask= "/content/yolov5/body-pix-not-armed/person/output-colored-mask"
outputPathMergeImages = "data-set-person-not-armed"
mergeImages = MergeImages(pathOriginal,"jpg",pathColoredMask,"jpg",outputPathMergeImages)

In [27]:
mergeImages.processImages()

Saved merge image as data-set-person-not-armed/0008681212.jpg
Saved merge image as data-set-person-not-armed/008987685.jpg
Saved merge image as data-set-person-not-armed/031972085.jpg
Saved merge image as data-set-person-not-armed/003373011.jpg
Saved merge image as data-set-person-not-armed/012626474.jpg
Saved merge image as data-set-person-not-armed/003218440.jpg
Saved merge image as data-set-person-not-armed/009124239.jpg
Saved merge image as data-set-person-not-armed/thirteen-j-NUAazpP3KIw-unsplash.jpg
Saved merge image as data-set-person-not-armed/gianandrea-villa-t4cSB-InqGY-unsplash.jpg
Saved merge image as data-set-person-not-armed/gift-habeshaw-ImFZSnfobKk-unsplash.jpg
Saved merge image as data-set-person-not-armed/000618382.jpg
Saved merge image as data-set-person-not-armed/010468957.jpg
Saved merge image as data-set-person-not-armed/001444210.jpg
Saved merge image as data-set-person-not-armed/010456784.jpg
Saved merge image as data-set-person-not-armed/011786011.jpg
Saved mer

In [28]:
!tar -czvf data-set-person-not-armed.tar.gz "data-set-person-not-armed"

data-set-person-not-armed/
data-set-person-not-armed/0008681212.jpg
data-set-person-not-armed/008987685.jpg
data-set-person-not-armed/031972085.jpg
data-set-person-not-armed/003373011.jpg
data-set-person-not-armed/012626474.jpg
data-set-person-not-armed/003218440.jpg
data-set-person-not-armed/009124239.jpg
data-set-person-not-armed/thirteen-j-NUAazpP3KIw-unsplash.jpg
data-set-person-not-armed/gianandrea-villa-t4cSB-InqGY-unsplash.jpg
data-set-person-not-armed/gift-habeshaw-ImFZSnfobKk-unsplash.jpg
data-set-person-not-armed/000618382.jpg
data-set-person-not-armed/010468957.jpg
data-set-person-not-armed/001444210.jpg
data-set-person-not-armed/010456784.jpg
data-set-person-not-armed/011786011.jpg
data-set-person-not-armed/016056238.jpg
data-set-person-not-armed/036484106.jpg
data-set-person-not-armed/005784268.jpg
data-set-person-not-armed/pexels-matheus-bertelli-1906157.jpg
data-set-person-not-armed/004011658.jpg
data-set-person-not-armed/006026739.jpg
data-set-person-not-armed/033804242

## Creacion proyecto armas cortas mascara de bits

In [29]:
!rm -rf data-set-person-not-armed-bitmask
!mkdir data-set-person-not-armed-bitmask

In [32]:
pathOriginal ="/content/yolov5/detect-not-armed-humans/exp/crops/person"
pathBitMask= "/content/yolov5/body-pix-not-armed/person/output-mask"
outputPathMergeImages = "data-set-person-not-armed-bitmask"
mergeImages = MergeImages(pathOriginal,"jpg",pathBitMask,"jpg",outputPathMergeImages)

In [33]:
mergeImages.processImages()

Saved merge image as data-set-person-not-armed-bitmask/0008681212.jpg
Saved merge image as data-set-person-not-armed-bitmask/008987685.jpg
Saved merge image as data-set-person-not-armed-bitmask/031972085.jpg
Saved merge image as data-set-person-not-armed-bitmask/003373011.jpg
Saved merge image as data-set-person-not-armed-bitmask/012626474.jpg
Saved merge image as data-set-person-not-armed-bitmask/003218440.jpg
Saved merge image as data-set-person-not-armed-bitmask/009124239.jpg
Saved merge image as data-set-person-not-armed-bitmask/thirteen-j-NUAazpP3KIw-unsplash.jpg
Saved merge image as data-set-person-not-armed-bitmask/gianandrea-villa-t4cSB-InqGY-unsplash.jpg
Saved merge image as data-set-person-not-armed-bitmask/gift-habeshaw-ImFZSnfobKk-unsplash.jpg
Saved merge image as data-set-person-not-armed-bitmask/000618382.jpg
Saved merge image as data-set-person-not-armed-bitmask/010468957.jpg
Saved merge image as data-set-person-not-armed-bitmask/001444210.jpg
Saved merge image as data-s

## Compresion Archivos

In [34]:
!rm -rf data-set-person-not-armed-bitmask.tar.gz
!tar -czvf data-set-person-not-armed-bitmask.tar.gz "data-set-person-not-armed-bitmask"

data-set-person-not-armed-bitmask/
data-set-person-not-armed-bitmask/0008681212.jpg
data-set-person-not-armed-bitmask/008987685.jpg
data-set-person-not-armed-bitmask/031972085.jpg
data-set-person-not-armed-bitmask/003373011.jpg
data-set-person-not-armed-bitmask/012626474.jpg
data-set-person-not-armed-bitmask/003218440.jpg
data-set-person-not-armed-bitmask/009124239.jpg
data-set-person-not-armed-bitmask/thirteen-j-NUAazpP3KIw-unsplash.jpg
data-set-person-not-armed-bitmask/gianandrea-villa-t4cSB-InqGY-unsplash.jpg
data-set-person-not-armed-bitmask/gift-habeshaw-ImFZSnfobKk-unsplash.jpg
data-set-person-not-armed-bitmask/000618382.jpg
data-set-person-not-armed-bitmask/010468957.jpg
data-set-person-not-armed-bitmask/001444210.jpg
data-set-person-not-armed-bitmask/010456784.jpg
data-set-person-not-armed-bitmask/011786011.jpg
data-set-person-not-armed-bitmask/016056238.jpg
data-set-person-not-armed-bitmask/036484106.jpg
data-set-person-not-armed-bitmask/005784268.jpg
data-set-person-not-armed

## Cargar en drive resultados

In [35]:
import os
from google.colab import drive
ROOT_FOLDER='/content/'
ROOT_DRIVE_FOLDER=ROOT_FOLDER+'drive/'
drive.mount(ROOT_DRIVE_FOLDER)

Mounted at /content/drive/


In [36]:
!cp --suffix=_v_$(date +"%y-%m-%d--%T")  ./data-set-person-not-armed.tar.gz /content/drive/MyDrive/tmf-person-data/data_set_processed/merge-dataset

In [37]:
!cp --suffix=_v_$(date +"%y-%m-%d--%T")  ./data-set-person-not-armed.tar.gz /content/drive/MyDrive/tmf-person-data/data_set_processed/merge-dataset